In [ ]:
!pip install -U hopsworks --quiet

In [3]:
from math import radians

import numpy as np
import pandas as pd

import transactions_fraud

import great_expectations
# Mute warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [ ]:
path = kagglehub.dataset_download("gpreda/bbc-news")
print("Path to dataset files:", path #"/home/jupyter/.cache/kagglehub/datasets/gpreda/bbc-news/versions/100" 
shutil.move(path, './')

In [15]:
def get_backfill():
    file_path = './bbc_news.csv'
    df = pd.read_csv(file_path)
    df = df[['pubDate', 'title']]
    df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S GMT')
    # as 'YYYY-MM-DD'
    df['pubDate'] =  pd.to_datetime(df['pubDate'].dt.strftime('%Y-%m-%d'))
    df2 = df[df['pubDate'] > pd.to_datetime('2023-11-30')]
    print(df2.head())
    print(df.tail())
    print(len(df))
    return df2
df_headlines = get_backfill()

import hopsworks
KEY = "wOWQmzzHeITT5wlJ.FkOButuQ3XpDXSUO1LnTuaNfD7SPWolfF1hateoistsLFFsBF7upULn5z6qKPOIB"
project = hopsworks.login(api_key_value=KEY)
fs = project.get_feature_store()
headlines_fg = fs.get_or_create_feature_group(
    name='headlines_new',
    description='Date of headline and the headline from bbc news',
    version=1,
    primary_key=['title'],
    online_enabled=True 
)
print(df_headlines)

headlines_fg.insert(df_headlines)

         pubDate                                              title
25931 2023-12-01  'I’m not ready to lose hope’: The hostages sti...
25932 2023-12-02  COP28: Huge rise in oil drilling expected from...
25933 2023-12-01  England 3-2 Netherlands: Ella Toone scores lat...
25934 2023-12-01  University Challenge: Christmas episode axed a...
25935 2023-12-01  King's tie features Greek flag after Elgin Mar...
         pubDate                                              title
41961 2024-11-29  Ditcheva first British woman to win MMA world ...
41962 2024-11-29  UK's first Dignitas widow feels ‘wonderful’ af...
41963 2024-11-29  Tight three-way race in Irish general election...
41964 2024-11-29  Could police have handled Skye shooting differ...
41965 2024-11-29        Angry doctors owed thousands refuse to work
41966
2024-12-02 12:08:13,273 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-02 12:08:13,276 INFO: Initializing external client
2024-12-02 12:08

Uploading Dataframe: 100.00% |██████████| Rows 16003/16003 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: headlines_new_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1163414/jobs/named/headlines_new_1_offline_fg_materialization/executions


(Job('headlines_new_1_offline_fg_materialization', 'SPARK'), None)

In [17]:
trans_fg = fs.get_feature_group(
    name='headlines_new', 
    version=1,
)
selected_features = trans_fg.select_all()

In [18]:
feature_view = fs.get_or_create_feature_view(
    name='headlines_new',
    version=1,
    query=selected_features,
    logging_enabled=True
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1163414/fs/1154117/fv/headlines_new/version/1


In [19]:
feature_df = feature_view.training_data(
    description = 'Headlines',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.45s) 


In [20]:
print(feature_df)

(                         pubdate  \
0      2024-02-25 00:00:00+00:00   
1      2024-05-17 00:00:00+00:00   
2      2024-11-21 00:00:00+00:00   
3      2024-01-03 00:00:00+00:00   
4      2024-06-10 00:00:00+00:00   
...                          ...   
14684  2024-06-03 00:00:00+00:00   
14685  2024-06-07 00:00:00+00:00   
14686  2024-05-01 00:00:00+00:00   
14687  2024-10-17 00:00:00+00:00   
14688  2024-09-19 00:00:00+00:00   

                                                   title  
0      Man sets himself on fire outside US Israeli em...  
1                WSL title race 'all comes down to this'  
2      Son of IRA murder victim calls Disney drama 'h...  
3      Statistics watchdog to examine government asyl...  
4      Raducanu in 'really fit place' as grass season...  
...                                                  ...  
14684  Dog fighting group jailed after illegal bouts ...  
14685          Eminem claims first number one since 2020  
14686   Arizona Senate votes to rep